In [1]:
import numpy as np
import pandas as pd
from starfish.plot import *

In [2]:
from starfish.set import settings
settings.cosmo_value

[0.32, 0.049, 0.67, 0.96, 0.81, 0.06, 3.044, -1, 0]

In [3]:
def var_filter(fisher,lcdm,neff_fixed):
    df=fisher.copy()
    if lcdm:
        df=df.drop('w0',axis=0)
        df=df.drop('w0',axis=1)
        df=df.drop('wa',axis=0)
        df=df.drop('wa',axis=1)
    if neff_fixed:
        df=df.drop('N_eff',axis=0)
        df=df.drop('N_eff',axis=1)

    return df

# nuisance parameters of different tracers/surveys should be independent of each other

In [4]:
JPAS_multi=var_filter(pd.read_excel('../data/jpas_all/valid.xlsx',header=0,index_col=0),lcdm=False,neff_fixed=False)
cov=pd.DataFrame(np.linalg.inv(JPAS_multi.values),index=JPAS_multi.index,columns=JPAS_multi.columns)
JPAS_multi_valid=np.linalg.inv(cov.iloc[:settings.cosmo_num,:settings.cosmo_num].values)

In [5]:
PFS_ELG=var_filter(pd.read_excel('../data/PFS/fisher.xlsx',header=0,index_col=0),lcdm=False,neff_fixed=False)
cov=pd.DataFrame(np.linalg.inv(PFS_ELG.values),index=PFS_ELG.index,columns=PFS_ELG.columns)
PFS_ELG_valid=np.linalg.inv(cov.iloc[:settings.cosmo_num,:settings.cosmo_num].values)

# J-PAS multitracer + PFS ELG

In [6]:
fisher1=pd.DataFrame(JPAS_multi_valid+PFS_ELG_valid,index=settings.var_name,columns=settings.var_name)
fisher1.to_excel('../data/combined/pelg+jmul-w0wacdm-free.xlsx')
fisher1

,omega_m0,omega_b0,h,n_s,sigma_8,m_nu,N_eff,w0,wa
omega_m0,543831.929978,-9.858576e+05,74888.073197,221829.070537,-10062.167937,-22067.404750,-8468.205857,-19574.613427,-1859.913506
omega_b0,-985857.647127,2.505464e+06,-410464.011839,-423082.660153,44839.019531,42500.389546,21380.711460,10982.376326,137.442784
h,74888.073197,-4.104640e+05,285152.296991,48556.944450,-98803.121008,-5392.025981,-4066.114502,16151.809616,5443.117187
n_s,221829.070537,-4.230827e+05,48556.944450,103838.218472,-17728.695611,-10113.712011,-3841.449518,-7218.068523,-727.985857
sigma_8,-10062.167937,4.483902e+04,-98803.121007,-17728.695611,82575.267173,3933.386348,457.330952,-1790.426178,-1869.841147
m_nu,-22067.404750,4.250039e+04,-5392.025981,-10113.712011,3933.386348,1160.792168,371.743250,923.706524,84.400591
N_eff,-8468.205857,2.138071e+04,-4066.114502,-3841.449518,457.330952,371.743250,198.359135,63.607594,-6.059102
w0,-19574.613427,1.098238e+04,16151.809616,-7218.068523,-1790.426178,923.706524,63.607594,3290.396906,655.475685
wa,-1859.913506,1.374428e+02,5443.117187,-727.985857,-1869.841147,84.400591,-6.059102,655.475685,196.195825


In [7]:
cov1=pd.DataFrame(np.linalg.inv(fisher1.values),index=settings.var_name,columns=settings.var_name)
cov1

,omega_m0,omega_b0,h,n_s,sigma_8,m_nu,N_eff,w0,wa
omega_m0,0.000164,0.000026,-0.000044,-0.000146,-0.000236,0.002019,-0.003218,0.001049,-0.004507
omega_b0,0.000026,0.000012,0.000007,-0.000032,-0.000010,-0.000048,-0.000592,0.000189,-0.000809
h,-0.000044,0.000007,0.000161,0.000022,0.000283,-0.002846,0.005799,-0.000136,-0.000255
n_s,-0.000146,-0.000032,0.000022,0.000255,0.000148,-0.000446,0.003495,-0.000929,0.003777
sigma_8,-0.000236,-0.000010,0.000283,0.000148,0.000715,-0.007084,0.011882,-0.001267,0.004936
m_nu,0.002019,-0.000048,-0.002846,-0.000446,-0.007084,0.080542,-0.114024,0.008171,-0.036516
N_eff,-0.003218,-0.000592,0.005799,0.003495,0.011882,-0.114024,0.309502,-0.012701,0.036270
w0,0.001049,0.000189,-0.000136,-0.000929,-0.001267,0.008171,-0.012701,0.008879,-0.035526
wa,-0.004507,-0.000809,-0.000255,0.003777,0.004936,-0.036516,0.036270,-0.035526,0.166589


In [8]:
name_width=max([len(v) for v in settings.var_name])
error_dict=dict(zip(settings.var_name,np.sqrt(np.diag(cov1.values))))
print('1-sigma errors:')
for key in error_dict.keys():
    print(f'-- {key.rjust(name_width)}: {error_dict[key]:.1e}')

1-sigma errors:
-- omega_m0: 1.3e-02
-- omega_b0: 3.4e-03
--        h: 1.3e-02
--      n_s: 1.6e-02
--  sigma_8: 2.7e-02
--     m_nu: 2.8e-01
--    N_eff: 5.6e-01
--       w0: 9.4e-02
--       wa: 4.1e-01


# + CMB-S4 + LiteBIRD

In [9]:
cmb_fisher=var_filter(pd.read_excel('../data/CMB/all.xlsx',header=0,index_col=0),lcdm=False,neff_fixed=False)
cmb_fisher

,omega_m0,omega_b0,h,n_s,sigma_8,m_nu,N_eff,tau,w0,wa
omega_m0,1.851288e+08,-6.928981e+08,1.371057e+08,-4.740936e+06,-5.782434e+07,-1.244715e+07,-6.589565e+06,4.485034e+07,1.850759e+06,113623.776249
omega_b0,-6.928981e+08,3.023402e+09,-4.583542e+08,3.206098e+07,2.226094e+08,4.714503e+07,2.328230e+07,-1.714219e+08,-7.598367e+06,-577001.097623
h,1.371057e+08,-4.583542e+08,1.125702e+08,-1.894718e+06,-3.817385e+07,-7.840066e+06,-5.484485e+06,2.962660e+07,3.552608e+06,706066.242382
n_s,-4.740936e+06,3.206098e+07,-1.894718e+06,1.046895e+06,1.665063e+06,3.149959e+05,1.125701e+05,-1.284039e+06,-1.360686e+05,-25864.908419
sigma_8,-5.782434e+07,2.226094e+08,-3.817385e+07,1.665063e+06,2.236167e+07,5.252308e+06,1.591679e+06,-1.731189e+07,1.719659e+06,615137.948390
m_nu,-1.244715e+07,4.714503e+07,-7.840066e+06,3.149959e+05,5.252308e+06,1.282975e+06,2.946820e+05,-4.090356e+06,6.369820e+05,207844.867608
N_eff,-6.589565e+06,2.328230e+07,-5.484485e+06,1.125701e+05,1.591679e+06,2.946820e+05,2.884944e+05,-1.230550e+06,-3.141717e+05,-74443.627745
tau,4.485034e+07,-1.714219e+08,2.962660e+07,-1.284039e+06,-1.731189e+07,-4.090356e+06,-1.230550e+06,1.374361e+07,-1.374592e+06,-488745.187357
w0,1.850759e+06,-7.598367e+06,3.552608e+06,-1.360686e+05,1.719659e+06,6.369820e+05,-3.141717e+05,-1.374592e+06,1.329266e+06,371850.395605
wa,1.136238e+05,-5.770011e+05,7.060662e+05,-2.586491e+04,6.151379e+05,2.078449e+05,-7.444363e+04,-4.887452e+05,3.718504e+05,104924.460938


In [10]:
cmb_cov=pd.DataFrame(np.linalg.inv(cmb_fisher.values),index=cmb_fisher.index,columns=cmb_fisher.columns)
cmb_cov_marg=cmb_cov.drop('tau',axis=0)
cmb_cov_marg=cmb_cov_marg.drop('tau',axis=1)
cmb_fisher_marg=pd.DataFrame(np.linalg.inv(cmb_cov_marg),index=settings.var_name,columns=settings.var_name)
cmb_fisher_marg

,omega_m0,omega_b0,h,n_s,sigma_8,m_nu,N_eff,w0,wa
omega_m0,3.876607e+07,-1.334868e+08,4.042346e+07,-5.506558e+05,-1.329430e+06,9.011528e+05,-2.573839e+06,6.336546e+06,1.708575e+06
omega_b0,-1.334868e+08,8.852823e+08,-8.882606e+07,1.604536e+07,6.680925e+06,-3.873349e+06,7.933847e+06,-2.474344e+07,-6.673045e+06
h,4.042346e+07,-8.882606e+07,4.870520e+07,8.732384e+05,-8.552429e+05,9.773665e+05,-2.831834e+06,6.515766e+06,1.759637e+06
n_s,-5.506558e+05,1.604536e+07,8.732384e+05,9.269296e+05,4.764654e+04,-6.715809e+04,-2.397812e+03,-2.644941e+05,-7.152742e+04
sigma_8,-1.329430e+06,6.680925e+06,-8.552429e+05,4.764654e+04,5.550676e+05,9.996606e+04,4.163926e+04,-1.182110e+04,-5.009973e+02
m_nu,9.011528e+05,-3.873349e+06,9.773665e+05,-6.715809e+04,9.996606e+04,6.560846e+04,-7.155261e+04,2.278777e+05,6.238514e+04
N_eff,-2.573839e+06,7.933847e+06,-2.831834e+06,-2.397812e+03,4.163926e+04,-7.155261e+04,1.783157e+05,-4.372473e+05,-1.182040e+05
w0,6.336546e+06,-2.474344e+07,6.515766e+06,-2.644941e+05,-1.182110e+04,2.278777e+05,-4.372473e+05,1.191783e+06,3.229677e+05
wa,1.708575e+06,-6.673045e+06,1.759637e+06,-7.152742e+04,-5.009973e+02,6.238514e+04,-1.182040e+05,3.229677e+05,8.754388e+04


In [11]:
fisher1_combined=fisher1+cmb_fisher_marg
fisher1_combined.to_excel('../data/combined/cmb+pelg+jmul-w0wacdm-free.xlsx')
cov1_combined=pd.DataFrame(np.linalg.inv(fisher1_combined),index=fisher1_combined.index,columns=fisher1_combined.columns)
cov1_combined

,omega_m0,omega_b0,h,n_s,sigma_8,m_nu,N_eff,w0,wa
omega_m0,9.623141e-06,7.715213e-07,-4.671052e-06,9.664859e-09,-1.467154e-05,0.000111,3.228303e-05,0.000091,-0.000405
omega_b0,7.715213e-07,1.755306e-07,-1.154973e-06,-4.851880e-08,-1.281353e-06,-0.000002,-2.121345e-06,0.000012,-0.000023
h,-4.671052e-06,-1.154973e-06,8.083214e-06,7.829283e-07,8.444622e-06,0.000017,3.272537e-05,-0.000070,0.000132
n_s,9.664859e-09,-4.851880e-08,7.829283e-07,5.229064e-06,8.822735e-07,-0.000016,5.510044e-05,0.000034,-0.000056
sigma_8,-1.467154e-05,-1.281353e-06,8.444622e-06,8.822735e-07,3.442800e-05,-0.000233,-5.436115e-07,-0.000156,0.000759
m_nu,1.108263e-04,-2.277862e-06,1.727880e-05,-1.559566e-05,-2.334181e-04,0.002963,3.455975e-04,0.000636,-0.006685
N_eff,3.228303e-05,-2.121345e-06,3.272537e-05,5.510044e-05,-5.436115e-07,0.000346,1.284574e-03,0.000485,-0.001705
w0,9.061291e-05,1.161179e-05,-7.008762e-05,3.408786e-05,-1.555328e-04,0.000636,4.845382e-04,0.002078,-0.006911
wa,-4.045643e-04,-2.256329e-05,1.315846e-04,-5.568959e-05,7.588436e-04,-0.006685,-1.704853e-03,-0.006911,0.031446


In [12]:
name_width=max([len(v) for v in settings.var_name])
error_dict=dict(zip(settings.var_name,np.sqrt(np.diag(cov1_combined.values))))
print('1-sigma errors:')
for key in error_dict.keys():
    print(f'-- {key.rjust(name_width)}: {error_dict[key]:.1e}')

1-sigma errors:
-- omega_m0: 3.1e-03
-- omega_b0: 4.2e-04
--        h: 2.8e-03
--      n_s: 2.3e-03
--  sigma_8: 5.9e-03
--     m_nu: 5.4e-02
--    N_eff: 3.6e-02
--       w0: 4.6e-02
--       wa: 1.8e-01


# CMB-S4 + LiteBIRD + PFS

In [13]:
PFS_combined=PFS_ELG_valid+cmb_fisher_marg
PFS_combined.to_excel('../data/combined/cmb+pelg-w0wacdm-free.xlsx')
cov_combined=pd.DataFrame(np.linalg.inv(PFS_combined),index=PFS_combined.index,columns=PFS_combined.columns)
cov_combined

,omega_m0,omega_b0,h,n_s,sigma_8,m_nu,N_eff,w0,wa
omega_m0,3.090398e-05,3.839308e-06,-2.548702e-05,8.839882e-07,-0.000031,0.000103,5.602559e-05,0.000171,-0.000425
omega_b0,3.839308e-06,6.609005e-07,-4.439896e-06,1.132817e-07,-0.000004,-0.000008,4.483143e-07,0.000024,-0.000017
h,-2.548702e-05,-4.439896e-06,3.040674e-05,2.682313e-07,0.000026,0.000050,2.301132e-05,-0.000151,0.000101
n_s,8.839882e-07,1.132817e-07,2.682313e-07,9.285408e-06,0.000004,-0.000059,1.092863e-04,0.000043,0.000025
sigma_8,-3.133442e-05,-3.733385e-06,2.553429e-05,3.792796e-06,0.000058,-0.000302,4.088482e-05,-0.000253,0.001021
m_nu,1.032586e-04,-8.027931e-06,5.011623e-05,-5.935782e-05,-0.000302,0.004136,-1.658384e-04,0.000771,-0.009699
N_eff,5.602559e-05,4.483143e-07,2.301132e-05,1.092863e-04,0.000041,-0.000166,2.076510e-03,0.000584,-0.000664
w0,1.705997e-04,2.381725e-05,-1.511338e-04,4.284237e-05,-0.000253,0.000771,5.837377e-04,0.003004,-0.009284
wa,-4.246836e-04,-1.674734e-05,1.008627e-04,2.535120e-05,0.001021,-0.009699,-6.644133e-04,-0.009284,0.045277


In [14]:
name_width=max([len(v) for v in settings.var_name])
error_dict=dict(zip(settings.var_name,np.sqrt(np.diag(cov_combined.values))))
print('1-sigma errors:')
for key in error_dict.keys():
    print(f'-- {key.rjust(name_width)}: {error_dict[key]:.1e}')

1-sigma errors:
-- omega_m0: 5.6e-03
-- omega_b0: 8.1e-04
--        h: 5.5e-03
--      n_s: 3.0e-03
--  sigma_8: 7.6e-03
--     m_nu: 6.4e-02
--    N_eff: 4.6e-02
--       w0: 5.5e-02
--       wa: 2.1e-01
